In [1]:
import multiprocessing
import os
from re import A
from typing import ItemsView

from py import log
os.environ['PYTHONDONTWRITEBYTECODE'] = '1'
from xtquant import xttrader
from xtquant import xtdata
from xtquant.xttrader import XtQuantTrader, XtQuantTraderCallback
from xtquant.xttype import StockAccount
from xtquant import xtconstant
import pandas_market_calendars as mcal
import pandas as pd
import akshare as ak

d:\anaconda3\envs\python3.8\lib\site-packages\exchange_calendars\exchange_calendar.py:2345: FutureWarning: 'T' is deprecated and will be removed in a future version. Please use 'min' instead of 'T'.
  align: pd.Timedelta | str = pd.Timedelta(1, "T"),


In [2]:
def caculate_returns(returns_df, row):
    cumulative_returns = (1 + returns_df[row]).cumprod()

    # 计算最大回撤
    cumulative_max = cumulative_returns.cummax()
    drawdown = (cumulative_returns - cumulative_max) / cumulative_max
    max_drawdown = drawdown.min()

    # 计算夏普比率
    risk_free_rate = 0.0  # 假设无风险利率为0
    sharpe_ratio = (returns_df[row].mean() - risk_free_rate) / returns_df[row].std()

    # 计算总收益率
    total_return = cumulative_returns.iloc[-1] - 1

    # 计算波动率
    volatility = returns_df[row].std()

    # 计算最大收益
    max_return = returns_df[row].max()

    # 计算总盈亏
    total_profit_loss = cumulative_returns.iloc[-1] - cumulative_returns.iloc[0]

    print(f"最大回撤: {max_drawdown:.2%}")
    print(f"夏普比率: {sharpe_ratio:.2f}")
    print(f"总收益率: {total_return:.2%}")
    print(f"波动率: {volatility:.2%}")
    print(f"最大收益: {max_return:.2%}")
    print(f"总盈亏: {total_profit_loss:.2%}")

In [3]:
df = pd.read_csv(r'D:\workspace\TradeX\dwdx_roll_back_100.csv')

In [4]:
df.set_index('date', inplace=True)
df['cumulative_returns'] = (1 + df['return']).cumprod()
df['p_return'] = df['return'] * df['position']
df['p_cumulative_returns'] = (1 + df['p_return']).cumprod()
df['first_cumulative_returns'] = (1 + df['first_return']).cumprod()
df['p_first_return'] = df['first_return'] * df['position']
df['p_first_cumulative_returns'] = (1 + df['p_first_return']).cumprod()
df_g_zero = df[df['codes_num'] > 0]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.dates as mdates
def plot(df, column):
    
    # 设置中文字体
    mpl.rcParams['font.sans-serif'] = ['SimHei']  # 指定中文字体为 SimHei
    mpl.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

    # 假设这是你的DataFrame，其中包含收益

    # 绘制收益曲线
    # 将日期列转换为日期类型
    # df['date'] = pd.to_datetime(df['date'])

    # # 设置日期列为索引
    # df.set_index('date', inplace=True)
    df.index = pd.to_datetime(df.index)
    # 绘制收益曲线
    plt.figure(figsize=(10, 6))  # 设置图表大小
    plt.plot(df.index, df[column], label='Return')
    plt.title('收益曲线')
    plt.xlabel('日期')
    plt.ylabel('收益率')
    plt.legend()  # 显示图例
    plt.grid(True)  # 显示网格线

    # plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    # plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=10))

    plt.show()

In [6]:
df

,code,name,return,max_return,first_return,position,codes_num,cumulative_returns,p_return,p_cumulative_returns,first_cumulative_returns,p_first_return,p_first_cumulative_returns
date,,,,,,,,,,,,,
2024-08-27,NaN,low,0.000000,0.000000,0.000000,0.325326,0,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000
2024-08-28,NaN,low,0.000000,0.000000,0.000000,0.311160,0,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000
2024-08-29,603038,low,-0.008906,0.000000,-0.008906,0.418266,1,0.991094,-0.003725,0.996275,0.991094,-0.003725,0.996275
2024-08-30,NaN,low,0.000000,0.000000,0.000000,0.667236,0,0.991094,0.000000,0.996275,0.991094,0.000000,0.996275
2024-09-02,NaN,low,0.000000,0.000000,0.000000,0.469144,0,0.991094,0.000000,0.996275,0.991094,0.000000,0.996275
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-17,NaN,low,0.000000,0.000000,0.000000,0.560071,0,1.083122,0.000000,1.198874,1.001967,0.000000,1.202650
2025-01-20,NaN,low,0.000000,0.000000,0.000000,0.664101,0,1.083122,0.000000,1.198874,1.001967,0.000000,1.202650
2025-01-21,002291,low,-0.019461,0.000000,-0.019461,0.363078,1,1.062044,-0.007066,1.190403,0.982467,-0.007066,1.194152


In [7]:
caculate_returns(df_g_zero, 'return')

最大回撤: -20.32%
夏普比率: 0.04
总收益率: 2.72%
波动率: 6.11%
最大收益: 16.11%
总盈亏: 3.61%


In [8]:
caculate_returns(df_g_zero, 'p_return')

最大回撤: -15.07%
夏普比率: 0.14
总收益率: 18.06%
波动率: 4.09%
最大收益: 14.37%
总盈亏: 18.43%


In [9]:
caculate_returns(df_g_zero, 'first_return')

最大回撤: -21.46%
夏普比率: 0.01
总收益率: -4.98%
波动率: 6.28%
最大收益: 16.11%
总盈亏: -4.09%


In [10]:
caculate_returns(df_g_zero, 'p_first_return')

最大回撤: -14.75%
夏普比率: 0.14
总收益率: 18.43%
波动率: 4.16%
最大收益: 14.37%
总盈亏: 18.80%


In [11]:
plot(df, 'return')

KeyError: 'column'

<Figure size 1000x600 with 0 Axes>

In [ ]:
plot(df, 'p_return')

In [ ]:
plot(df, 'first_return')

In [ ]:
plot(df, 'p_first_return')